In [1]:
import configparser

# Read the configuration file
config = configparser.ConfigParser()
config.read('../config.ini')

# Access the API token from the configuration file
huggingfacehub_api_token = config.get('huggingface', 'api_token')
# Set your API Key from OpenAI
openai_api_key = config.get('openai', 'api_key')


In [2]:
#basics chain sequence using LCEL
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(openai_api_key=openai_api_key)
prompt = ChatPromptTemplate.from_template("You are a skilled poet. Write a haiku about the following topic: {topic}")

# Define the chain using LCEL
chain = prompt | model

# Invoke the chain with any topic
print(chain.invoke({"topic": "Becode"}))

content='Coding with BeCode\nLearning skills for the future\nTech world at our feet' response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 24, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-30beb457-e712-4c85-94c8-1ed7340ec38d-0'


In [5]:
# same but with passthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

# Create the retriever and model
vectorstore = Chroma.from_texts(["the fish is pink like a sun."], embedding=OpenAIEmbeddings(openai_api_key=openai_api_key))
retriever = vectorstore.as_retriever()
model = ChatOpenAI(openai_api_key=openai_api_key, temperature=0)

template = """Answer the question based on the context:{context}. Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

# Create the chain and run it
chain = (
  {"context": retriever, "question":RunnablePassthrough()}
  | prompt
  | model)

chain.invoke("how the fish looks like, in 2 words?")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


AIMessage(content='Pink sun', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 78, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-db9434b5-7495-4af5-8e50-ab8b741f619c-0')

In [10]:
# longer chain example 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser




coding_prompt = PromptTemplate.from_template(
    """Write Python code to loop through the following list, printing each element: {list}""")
validate_prompt = PromptTemplate.from_template(
    """Consider the following Python code: {answer} If it doesn't use a list comprehension, update it to use one. If it does use a list comprehension, return the original code without explanation:""")

llm = ChatOpenAI(openai_api_key=openai_api_key)

# Create the sequential chain
chain = ({"answer": coding_prompt | llm |StrOutputParser()}
         | validate_prompt
         | llm 
         | StrOutputParser() )

# Invoke the chain with the user's question
chain.invoke({"list": "[3, 1, 4, 1]"})

'```python\nmy_list = [3, 1, 4, 1]\n\n[element for element in my_list]\n```'

In [11]:
# full integrated chain chatbot like 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser




# Make ceo_response available for other chains
ceo_response = (
    ChatPromptTemplate.from_template("You are a CEO. Describe the most lucrative consumer product addressing the following consumer need in one sentence: {input}.")
    | ChatOpenAI(openai_api_key=openai_api_key)
    | {"ceo_response": RunnablePassthrough() | StrOutputParser()}
)

advisor_response = (
    ChatPromptTemplate.from_template("You are a strategic adviser. Briefly map the outline and business plan for {ceo_response} in 3 key steps.")
    | ChatOpenAI(openai_api_key=openai_api_key)
    | StrOutputParser()
)

overall_response = (
    ChatPromptTemplate.from_messages(
        [
            ("human", "CEO response:\n{ceo_response}\n\nAdvisor response:\n{advisor_response}"),
            ("system", "Generate a final response including the CEO's response, the advisor response, and a summary of the business plan in one sentence."),
        ]
    )
    | ChatOpenAI(openai_api_key=openai_api_key)
    | StrOutputParser()
)

# Create a chain to insert the outputs from the other chains into overall_response
business_idea_chain = (
    {"ceo_response": ceo_response, "advisor_response": advisor_response}
    | overall_response
    | ChatOpenAI(openai_api_key=openai_api_key)
    | StrOutputParser()
)

print(business_idea_chain.invoke({"input": "Typing on mobile touchscreens is slow.", "ceo_response": "", "advisor_response": ""}))

With our app, users can type faster and more accurately than ever before, thanks to our advanced predictive text technology and customizable keyboard layouts. We have conducted extensive market research to understand the needs and preferences of mobile users, and have tailored our app to meet those needs.

Our business plan outlines a clear path to profitability, with multiple revenue streams including in-app purchases, premium features, and partnerships with mobile device manufacturers. We have also developed a comprehensive marketing strategy to drive user acquisition and retention, including targeted advertising, social media campaigns, and influencer partnerships.

Through strategic implementation of our app, we aim to achieve widespread adoption and become the go-to virtual keyboard for mobile users worldwide. With a focus on user experience, innovation, and scalability, we are confident that our app will revolutionize typing on mobile touchscreens and disrupt the virtual keyboard